# load packages

In [ ]:
dir <- 'processed_data/peak_distribution/'

cores=10

In [16]:
suppressPackageStartupMessages({
    library(tidyverse)
    library(parallel)
    library(progress)
    library(qs)
})

In [19]:
source('../../source/basic.r')

In [17]:
psize <- function (w = 10, h = 10) 
{
    options(repr.plot.width = w, repr.plot.height = h)
}

# aggregate all peak regions

In [18]:
fun_addMat <- function(i){
    
    mat_out <-  mat[[i]]%>%full_join(mat_add[[i]],by=c('site','value'))%>%
                mutate(name=ifelse(is.na(name.x),name.y,
                               ifelse(is.na(name.y),name.x,
                                      paste0(name.x,',',name.y))))%>%
                # mutate(num=str_count(name,',')+1)%>%  ## add to the last dataframe
                select(site,value,name)
    return(mat_out)
    
}

In [5]:
name.list <-  list.files(dir,'*rds')%>%gsub('.rds','',.)

In [6]:
mat <- readRDS(paste0(dir,name.list[1],'.rds'))%>%
        lapply(function(x){x%>%cbind('name'=1)})

In [7]:
options(warn=2)
# change the level of warning 
# to stop the loop, when there's error in any cores
mat <- readRDS(paste0(dir,name.list[1],'.rds'))%>%
        lapply(function(x){x%>%cbind('name'=1)})
start_time <- Sys.time()
for(i in 2:length(name.list)){
    name = name.list[i]
    mat_add <- readRDS(paste0(dir,name,'.rds'))%>%
                lapply(function(x){x%>%cbind('name'=i)})
    if((nrow(mat_add[[24]])>10)){
        mat <- mclapply(1:24,fun_addMat,mc.cores= cores)
    }else{
        # 这里指没有Y染色体的情况，只需要改变chr1-chrY即可
        mat[1:23] <- mclapply(1:23,fun_addMat,mc.cores= cores)
    }
    if((i%%20)==1){
        message(i)
        end_time <- Sys.time()
        message((end_time-start_time))
        start_time <- Sys.time()
    }
    
}
# options(warn=1)

In [ ]:
mat2 <- mclapply(1:24,
                function(x){mat[[x]]%>%mutate(num=str_count(name,',')+1)},
               mc.cores= cores)

# split the aggregated data to PORs

In [ ]:
# split part

split_fun <- function(i){
    a = mat2[[i]]%>%arrange(site)
    a = a%>%mutate(intersect = c(1, (a$site)[c(2:nrow(a))] - a$site[c(1:(nrow(a)-1))]))

    aa = group_split(a,findInterval(1:nrow(a),which(a$intersect >1)))
    
    return(aa)
}

In [ ]:
mat_split <- mclapply(1:24, split_fun,mc.cores = cores)

# save POR

In [21]:
fun_getCountMat <- function(j,mat){
    
    mat[[j]]%>%select(site:intersect) %>%
    separate_rows(name,sep=',') %>%
    mutate(value = 1) %>%
    pivot_wider(names_from = name, values_fill = 0)%>%
    return

    
}


In [23]:
fun_all <- function(i){
    
    mat_chr=mat_split[[i]] #%>%.[1:2]
    
    mclapply(1:length(mat_chr),
             fun_getCountMat,
             mat=mat_chr,
             mc.cores = 2)%>%
    qsave(paste0('results/',i,'.qs'))
    #mclapply(1:2,fun_getCountMat,mc.cores = 20)%>%return
}

In [24]:
a <- mclapply(1:24,fun_all,mc.cores = 1)